# Aufmerksamkeitsmechanismen und Transformer

Ein wesentlicher Nachteil von rekurrenten Netzwerken ist, dass alle Wörter in einer Sequenz den gleichen Einfluss auf das Ergebnis haben. Dies führt zu suboptimaler Leistung bei Standard-LSTM-Encoder-Decoder-Modellen für Sequenz-zu-Sequenz-Aufgaben wie der Erkennung benannter Entitäten (Named Entity Recognition) und maschineller Übersetzung. In der Realität haben bestimmte Wörter in der Eingabesequenz oft mehr Einfluss auf die sequentiellen Ausgaben als andere.

Betrachten wir ein Sequenz-zu-Sequenz-Modell, wie es bei der maschinellen Übersetzung verwendet wird. Es wird durch zwei rekurrente Netzwerke implementiert, wobei ein Netzwerk (**Encoder**) die Eingabesequenz in einen versteckten Zustand komprimiert und ein anderes Netzwerk (**Decoder**) diesen versteckten Zustand in das übersetzte Ergebnis entfaltet. Das Problem bei diesem Ansatz ist, dass der Endzustand des Netzwerks Schwierigkeiten hat, sich an den Anfang eines Satzes zu erinnern, was zu einer schlechten Modellqualität bei langen Sätzen führt.

**Aufmerksamkeitsmechanismen** bieten eine Möglichkeit, den kontextuellen Einfluss jedes Eingabevektors auf jede Ausgabewahrscheinlichkeit des RNN zu gewichten. Dies wird durch die Erstellung von Abkürzungen zwischen den Zwischenzuständen des Eingabe-RNN und des Ausgabe-RNN umgesetzt. Auf diese Weise berücksichtigen wir bei der Generierung des Ausgabesymbols $y_t$ alle versteckten Eingabezustände $h_i$, mit unterschiedlichen Gewichtungskoeffizienten $\alpha_{t,i}$.

![Bild eines Encoder/Decoder-Modells mit einer additiven Aufmerksamkeits-Schicht](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*Das Encoder-Decoder-Modell mit additivem Aufmerksamkeitsmechanismus aus [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), zitiert aus [diesem Blogbeitrag](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Die Aufmerksamkeitsmatrix $\{\alpha_{i,j}\}$ repräsentiert den Grad, in dem bestimmte Eingabewörter bei der Generierung eines bestimmten Wortes in der Ausgabesequenz eine Rolle spielen. Unten ist ein Beispiel für eine solche Matrix dargestellt:

![Bild eines Beispiel-Alignments, gefunden von RNNsearch-50, aus Bahdanau - arviz.org](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*Abbildung aus [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Abb. 3)*

Aufmerksamkeitsmechanismen sind für einen Großteil des aktuellen oder nahezu aktuellen Stands der Technik in der Verarbeitung natürlicher Sprache verantwortlich. Das Hinzufügen von Aufmerksamkeit erhöht jedoch die Anzahl der Modellparameter erheblich, was zu Skalierungsproblemen bei RNNs führte. Eine zentrale Einschränkung bei der Skalierung von RNNs ist, dass die rekurrente Natur der Modelle es schwierig macht, das Training zu batchen und zu parallelisieren. In einem RNN muss jedes Element einer Sequenz in der Reihenfolge verarbeitet werden, was bedeutet, dass es nicht leicht parallelisiert werden kann.

Die Einführung von Aufmerksamkeitsmechanismen in Kombination mit dieser Einschränkung führte zur Entwicklung der heute bekannten und genutzten Transformer-Modelle, die den Stand der Technik darstellen, von BERT bis OpenGPT3.

## Transformer-Modelle

Anstatt den Kontext jeder vorherigen Vorhersage in den nächsten Evaluierungsschritt weiterzuleiten, verwenden **Transformer-Modelle** **Positionskodierungen** und **Aufmerksamkeit**, um den Kontext einer gegebenen Eingabe innerhalb eines bereitgestellten Textfensters zu erfassen. Das folgende Bild zeigt, wie Positionskodierungen mit Aufmerksamkeit den Kontext innerhalb eines bestimmten Fensters erfassen können.

![Animiertes GIF, das zeigt, wie die Auswertungen in Transformer-Modellen durchgeführt werden.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Da jede Eingabeposition unabhängig auf jede Ausgabeposition abgebildet wird, können Transformer besser parallelisieren als RNNs, was viel größere und ausdrucksstärkere Sprachmodelle ermöglicht. Jeder Aufmerksamkeitskopf kann verwendet werden, um unterschiedliche Beziehungen zwischen Wörtern zu lernen, was die nachgelagerten Aufgaben der Verarbeitung natürlicher Sprache verbessert.

## Einfaches Transformer-Modell erstellen

Keras enthält keine eingebaute Transformer-Schicht, aber wir können unsere eigene erstellen. Wie zuvor konzentrieren wir uns auf die Textklassifikation des AG-News-Datensatzes, aber es ist erwähnenswert, dass Transformer-Modelle bei schwierigeren NLP-Aufgaben die besten Ergebnisse zeigen.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Neue Schichten in Keras sollten die Klasse `Layer` unterklassen und die Methode `call` implementieren. Beginnen wir mit der **Positional Embedding**-Schicht. Wir werden [einige Code aus der offiziellen Keras-Dokumentation](https://keras.io/examples/nlp/text_classification_with_transformer/) verwenden. Wir nehmen an, dass wir alle Eingabesequenzen auf die Länge `maxlen` auffüllen.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Diese Schicht besteht aus zwei `Embedding`-Schichten: eine für das Einbetten von Tokens (wie wir es zuvor besprochen haben) und eine für die Positionen der Tokens. Die Positionen der Tokens werden als eine Sequenz natürlicher Zahlen von 0 bis `maxlen` mithilfe von `tf.range` erstellt und anschließend durch die Einbettungsschicht geleitet. Die beiden resultierenden Einbettungsvektoren werden dann addiert, wodurch eine positionsbezogene Einbettungsdarstellung der Eingabe mit der Form `maxlen`$\times$`embed_dim` entsteht.

Nun implementieren wir den Transformer-Block. Er wird die Ausgabe der zuvor definierten Einbettungsschicht übernehmen:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Der Transformer wendet `MultiHeadAttention` auf die positionskodierte Eingabe an, um den Aufmerksamkeitsvektor mit der Dimension `maxlen`$\times$`embed_dim` zu erzeugen, der dann mit der Eingabe gemischt und mittels `LayerNormalization` normalisiert wird.

> **Hinweis**: `LayerNormalization` ähnelt der `BatchNormalization`, die im Abschnitt *Computer Vision* dieses Lernpfads behandelt wurde. Sie normalisiert die Ausgaben der vorherigen Schicht für jede Trainingsprobe unabhängig, um sie in den Bereich [-1..1] zu bringen.

Die Ausgabe dieser Schicht wird anschließend durch ein `Dense` Netzwerk geleitet (in unserem Fall ein zweischichtiger Perzeptron), und das Ergebnis wird zur endgültigen Ausgabe hinzugefügt (die erneut normalisiert wird).


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer-Modelle

**BERT** (Bidirectional Encoder Representations from Transformers) ist ein sehr großes mehrschichtiges Transformer-Netzwerk mit 12 Schichten für *BERT-base* und 24 für *BERT-large*. Das Modell wird zunächst mit einem großen Textkorpus (Wikipedia + Bücher) mittels unüberwachtem Training vortrainiert (Vorhersage von maskierten Wörtern in einem Satz). Während des Vortrainings erlangt das Modell ein erhebliches Maß an Sprachverständnis, das anschließend mit anderen Datensätzen durch Feintuning genutzt werden kann. Dieser Prozess wird als **Transferlernen** bezeichnet.

![Bild von http://jalammar.github.io/illustrated-bert/](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

Es gibt viele Varianten von Transformer-Architekturen, darunter BERT, DistilBERT, BigBird, OpenGPT3 und mehr, die fein abgestimmt werden können.

Schauen wir uns an, wie wir ein vortrainiertes BERT-Modell verwenden können, um unser traditionelles Problem der Sequenzklassifikation zu lösen. Wir werden die Idee und etwas Code aus der [offiziellen Dokumentation](https://www.tensorflow.org/text/tutorials/classify_text_with_bert) übernehmen.

Um vortrainierte Modelle zu laden, verwenden wir **Tensorflow hub**. Zunächst laden wir den BERT-spezifischen Vektorisierer:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Es ist wichtig, dass Sie denselben Vektorisierer verwenden, mit dem das ursprüngliche Netzwerk trainiert wurde. Außerdem gibt der BERT-Vektorisierer drei Komponenten zurück:
* `input_word_ids`, eine Sequenz von Token-Nummern für den Eingabesatz
* `input_mask`, die zeigt, welcher Teil der Sequenz tatsächliche Eingaben enthält und welcher Teil Padding ist. Dies ist ähnlich der Maske, die von der `Masking`-Schicht erzeugt wird
* `input_type_ids` wird für Aufgaben im Bereich Sprachmodellierung verwendet und ermöglicht es, zwei Eingabesätze in einer Sequenz anzugeben.

Anschließend können wir den BERT-Feature-Extractor instanziieren:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Die BERT-Schicht liefert eine Reihe nützlicher Ergebnisse:
* `pooled_output` ist das Ergebnis des Mittelwerts aller Tokens in der Sequenz. Sie können es als eine intelligente semantische Einbettung des gesamten Netzwerks betrachten. Es entspricht der Ausgabe der `GlobalAveragePooling1D`-Schicht in unserem vorherigen Modell.
* `sequence_output` ist die Ausgabe der letzten Transformer-Schicht (entspricht der Ausgabe von `TransformerBlock` in unserem obigen Modell).
* `encoder_outputs` sind die Ausgaben aller Transformer-Schichten. Da wir ein 4-Schichten-BERT-Modell geladen haben (wie Sie wahrscheinlich aus dem Namen schließen können, der `4_H` enthält), gibt es 4 Tensoren. Der letzte davon ist identisch mit `sequence_output`.

Nun werden wir das End-to-End-Klassifikationsmodell definieren. Wir verwenden die *funktionale Modelldefinition*, bei der wir den Modelleingang definieren und dann eine Reihe von Ausdrücken angeben, um dessen Ausgabe zu berechnen. Außerdem werden wir die Gewichte des BERT-Modells nicht trainierbar machen und nur den finalen Klassifikator trainieren:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Trotz der Tatsache, dass es nur wenige trainierbare Parameter gibt, ist der Prozess ziemlich langsam, da der BERT-Feature-Extractor rechnerisch sehr aufwendig ist. Es scheint, dass wir keine angemessene Genauigkeit erreichen konnten, entweder aufgrund mangelnden Trainings oder fehlender Modellparameter.

Lassen Sie uns versuchen, die BERT-Gewichte zu entsperren und ebenfalls zu trainieren. Dies erfordert eine sehr kleine Lernrate und eine sorgfältigere Trainingsstrategie mit **Warmup**, unter Verwendung des **AdamW**-Optimierers. Wir werden das `tf-models-official`-Paket verwenden, um den Optimierer zu erstellen:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Wie Sie sehen, verläuft das Training ziemlich langsam – aber Sie könnten experimentieren und das Modell für ein paar Epochen (5-10) trainieren, um zu sehen, ob Sie im Vergleich zu den zuvor verwendeten Ansätzen das beste Ergebnis erzielen können.

## Huggingface Transformers-Bibliothek

Eine weitere sehr gängige (und etwas einfachere) Möglichkeit, Transformer-Modelle zu verwenden, ist das [HuggingFace-Paket](https://github.com/huggingface/), das einfache Bausteine für verschiedene NLP-Aufgaben bereitstellt. Es ist sowohl für Tensorflow als auch für PyTorch verfügbar, ein weiteres sehr beliebtes Framework für neuronale Netze.

> **Hinweis**: Wenn Sie nicht daran interessiert sind, zu sehen, wie die Transformers-Bibliothek funktioniert, können Sie zum Ende dieses Notebooks springen, da Sie nichts wesentlich anderes sehen werden als das, was wir oben gemacht haben. Wir werden die gleichen Schritte des Trainings eines BERT-Modells mit einer anderen Bibliothek und einem wesentlich größeren Modell wiederholen. Der Prozess beinhaltet daher ein ziemlich langes Training, sodass Sie vielleicht nur den Code durchsehen möchten.

Schauen wir uns an, wie unser Problem mit [Huggingface Transformers](http://huggingface.co) gelöst werden kann.


Das Erste, was wir tun müssen, ist, das Modell auszuwählen, das wir verwenden möchten. Neben einigen integrierten Modellen bietet Huggingface ein [Online-Modell-Repository](https://huggingface.co/models), in dem Sie viele weitere vortrainierte Modelle der Community finden können. All diese Modelle können geladen und verwendet werden, indem einfach der Modellname angegeben wird. Alle erforderlichen Binärdateien für das Modell werden automatisch heruntergeladen.

Manchmal müssen Sie Ihre eigenen Modelle laden. In diesem Fall können Sie das Verzeichnis angeben, das alle relevanten Dateien enthält, einschließlich der Parameter für den Tokenizer, der `config.json`-Datei mit den Modellparametern, der Binärgewichte usw.

Aus dem Modellnamen können wir sowohl das Modell als auch den Tokenizer instanziieren. Beginnen wir mit einem Tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Das `tokenizer`-Objekt enthält die `encode`-Funktion, die direkt verwendet werden kann, um Text zu kodieren:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Wir können auch den Tokenizer verwenden, um eine Sequenz auf eine Weise zu codieren, die für die Übergabe an das Modell geeignet ist, d. h. einschließlich der Felder `token_ids`, `input_mask` usw. Wir können auch angeben, dass wir Tensorflow-Tensoren möchten, indem wir das Argument `return_tensors='tf'` bereitstellen:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

In unserem Fall verwenden wir ein vortrainiertes BERT-Modell namens `bert-base-uncased`. *Uncased* bedeutet, dass das Modell nicht zwischen Groß- und Kleinschreibung unterscheidet.

Beim Training des Modells müssen wir eine tokenisierte Sequenz als Eingabe bereitstellen, und daher werden wir eine Datenverarbeitungspipeline entwerfen. Da `tokenizer.encode` eine Python-Funktion ist, werden wir denselben Ansatz wie in der letzten Einheit verwenden, indem wir sie mit `py_function` aufrufen:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Jetzt können wir das eigentliche Modell mit dem Paket `BertForSequenceClassification` laden. Dies stellt sicher, dass unser Modell bereits über eine erforderliche Architektur für die Klassifikation verfügt, einschließlich des endgültigen Klassifikators. Sie werden eine Warnmeldung sehen, die besagt, dass die Gewichte des endgültigen Klassifikators nicht initialisiert sind und das Modell ein Pre-Training erfordert – das ist völlig in Ordnung, denn genau das werden wir jetzt tun!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Wie Sie aus `summary()` sehen können, enthält das Modell fast 110 Millionen Parameter! Vermutlich möchten wir, wenn wir eine einfache Klassifikationsaufgabe auf einem relativ kleinen Datensatz durchführen wollen, die BERT-Basis-Schicht nicht trainieren:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Jetzt können wir mit dem Training beginnen!

> **Hinweis**: Das Training eines vollständigen BERT-Modells kann sehr zeitaufwendig sein! Daher werden wir es nur für die ersten 32 Batches trainieren. Dies dient lediglich dazu, zu zeigen, wie das Modelltraining eingerichtet wird. Wenn Sie daran interessiert sind, ein vollständiges Training auszuprobieren, entfernen Sie einfach die Parameter `steps_per_epoch` und `validation_steps` und bereiten Sie sich darauf vor, zu warten!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Wenn Sie die Anzahl der Iterationen erhöhen, lange genug warten und über mehrere Epochen trainieren, können Sie erwarten, dass die BERT-Klassifikation uns die beste Genauigkeit liefert! Das liegt daran, dass BERT die Struktur der Sprache bereits ziemlich gut versteht und wir nur den finalen Klassifikator feinabstimmen müssen. Allerdings ist BERT ein großes Modell, weshalb der gesamte Trainingsprozess viel Zeit in Anspruch nimmt und erhebliche Rechenleistung erfordert! (GPU, und vorzugsweise mehr als eine).

> **Hinweis:** In unserem Beispiel verwenden wir eines der kleinsten vortrainierten BERT-Modelle. Es gibt größere Modelle, die wahrscheinlich bessere Ergebnisse liefern.


## Fazit

In dieser Einheit haben wir uns mit sehr aktuellen Modellarchitekturen basierend auf **Transformers** beschäftigt. Wir haben sie für unsere Textklassifizierungsaufgabe angewendet, aber ebenso können BERT-Modelle für die Extraktion von Entitäten, das Beantworten von Fragen und andere NLP-Aufgaben genutzt werden.

Transformermodelle stellen den aktuellen Stand der Technik im Bereich NLP dar, und in den meisten Fällen sollten sie die erste Lösung sein, mit der Sie experimentieren, wenn Sie maßgeschneiderte NLP-Lösungen implementieren. Dennoch ist es äußerst wichtig, die grundlegenden Prinzipien von rekurrenten neuronalen Netzen, die in diesem Modul behandelt wurden, zu verstehen, wenn Sie fortgeschrittene neuronale Modelle entwickeln möchten.



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
